In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)

from sklearn.svm import SVC
from sklearn.utils import shuffle


# TODO: Change to larger training set

train_set = pd.read_csv('../Small Training Set.csv', header=None)

# train_set = pd.read_csv('../20 Percent Training Set.csv', header=None)
# train_set = pd.read_csv('../KDDTrain+.csv', header=None)

fn = pd.read_csv('../Field Names.csv', header=None)

fn2 = pd.DataFrame([['attack_type', 'symbolic'],['??','continuous']])
fn = fn.append(fn2, ignore_index=True)

cont_feats = fn.loc[fn[1] == 'continuous']
cont_feats = cont_feats.drop(columns=[1])

fn = fn.drop(columns=[1])

dfhd = fn[0].tolist()
train_set.columns = dfhd

cols = train_set.columns.tolist()
cols = cols[-2:-1] + cols[:-2] + cols[-1:]
train_set = train_set[cols]

att_types = train_set['attack_type']
att_types[att_types != 'normal'] = 'not normal'

train_set.sort_values('attack_type')

,attack_type,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,??
0,normal,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.0,0.05,0.00,20
543,normal,0,tcp,finger,SF,10,139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,126,5,0.04,0.03,0.01,0.00,0.00,0.0,0.00,0.00,21
544,normal,0,tcp,http,SF,311,488,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.00,0.00,0.00,104,255,1.00,0.00,0.01,0.04,0.01,0.0,0.00,0.00,21
545,normal,0,tcp,http,SF,281,1745,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,39,255,1.00,0.00,0.03,0.07,0.00,0.0,0.00,0.00,21
546,normal,0,tcp,http,SF,297,1285,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.00,0.00,0.00,8,255,1.00,0.00,0.12,0.04,0.00,0.0,0.00,0.00,21
548,normal,0,udp,domain_u,SF,43,72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,263,264,0.0,0.0,0.0,0.0,1.00,0.01,0.01,255,242,0.95,0.01,0.00,0.00,0.00,0.0,0.00,0.00,18
552,normal,0,tcp,http,SF,215,9399,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,23,0.0,0.0,0.0,0.0,1.00,0.00,0.13,79,255,1.00,0.00,0.01,0.05,0.00,0.0,0.00,0.00,21
554,normal,0,tcp,http,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,1.0,1.0,1.00,0.00,1.00,43,255,1.00,0.00,0.02,0.07,0.00,0.0,1.00,0.97,21
555,normal,0,udp,other,SF,516,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,151,151,0.0,0.0,0.0,0.0,1.00,0.00,0.00,151,151,1.00,0.00,1.00,0.00,0.00,0.0,0.00,0.00,17
560,normal,28,udp,domain_u,SF,44,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,0.0,0.0,0.0,0.0,0.33,0.67,0.40,101,98,0.97,0.02,0.01,0.00,0.00,0.0,0.00,0.00,20


In [3]:
# ENUMERATING SERVICE TYPE
import copy

service_list = train_set['service'].unique()

try:
    service_list2
except NameError:
    service_list2 = copy.deepcopy(service_list)    
    svc_enum_data = {'service': service_list}
    service_enumerations = pd.DataFrame(data=svc_enum_data)
    service_enumerations.to_csv('Service_enums.csv')

enumerated_service_list = [x for x in range(len(service_list))]


train_set = train_set.replace(service_list,enumerated_service_list)


In [4]:
# ENUMERATING PROTOCOL TYPE

prot_list = train_set['protocol_type'].unique()

try:
    prot_list2
except NameError:
    prot_list2 = copy.deepcopy(prot_list)    
    prot_enum_data = {'protocol_type': prot_list}
    prot_enumerations = pd.DataFrame(data=prot_enum_data)
    prot_enumerations.to_csv('Prot_enums.csv')

enumerated_prot_list = [x for x in range(len(prot_list))]


train_set = train_set.replace(prot_list,enumerated_prot_list)


In [5]:
# ENUMERATING FLAG TYPE

flag_list = train_set['flag'].unique()

try:
    flag_list2
except NameError:
    flag_list2 = copy.deepcopy(flag_list)    
    flag_enum_data = {'flag': flag_list}
    flag_enumerations = pd.DataFrame(data=flag_enum_data)
    flag_enumerations.to_csv('Flag_enums.csv')

enumerated_flag_list = [x for x in range(len(flag_list))]


train_set = train_set.replace(flag_list,enumerated_flag_list)

In [24]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200)

train_features = train_set.drop(columns=['attack_type'])
train_target = train_set['attack_type']

clf.fit(train_features, train_target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
test_set = pd.read_csv('../KDDTest+.csv', header=None)
test_set.columns = dfhd

test_set = test_set[cols]

test_set = test_set.replace(service_list2,enumerated_service_list)
test_set['service'] = test_set['service'].apply(lambda x: '-1' if type(x) is str else x)

test_set = test_set.replace(prot_list2, enumerated_prot_list)
test_set['protocol_type'] = test_set['protocol_type'].apply(lambda x: '-1' if type(x) is str else x)

test_set = test_set.replace(flag_list2, enumerated_flag_list)
test_set['flag'] = test_set['flag'].apply(lambda x: '-1' if type(x) is str else x)

# Mini test_set as not full training set used
test_set = shuffle(test_set)
test_set = test_set[:201]


test_features = test_set.drop(columns=['attack_type'])
test_target = test_set['attack_type']


pred_list = clf.predict(test_features)

np.mean( pred_list == test_target)

0.3781094527363184